In [1]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, roc_curve, auc, recall_score
from sklearn import preprocessing
from sklearn.datasets import load_iris

In [2]:
from IPython.display import SVG
from graphviz import Source
from IPython.display import display
from IPython.display import HTML
style = "<style>svg{width: 70% !important; height: 60% !important;} </style>" 
HTML(style)

In [3]:
submissions = pd.read_csv(".\submissions_data_train.csv")

In [25]:
events = pd.read_csv(".\event_data_train.csv")

In [42]:
events.head(4)

,step_id,timestamp,action,user_id,min_timestamp,date,day,user_time
0,32815,1434340848,viewed,17632,1434340848,2015-06-15 04:00:48,2015-06-15,17632_1434340848
1,32815,1434340848,passed,17632,1434340848,2015-06-15 04:00:48,2015-06-15,17632_1434340848
2,32815,1434340848,discovered,17632,1434340848,2015-06-15 04:00:48,2015-06-15,17632_1434340848
3,32811,1434340895,discovered,17632,1434340848,2015-06-15 04:01:35,2015-06-15,17632_1434340895


In [27]:
user_min_time = events.groupby("user_id", as_index = False) \
                .agg({"timestamp":"min"}) \
                .rename({"timestamp":"min_timestamp"}, axis = 1)

In [103]:
events['date'] = pd.to_datetime(events.timestamp, unit = 's')
events['day'] = events.date.dt.date

submissions['date'] = pd.to_datetime(submissions.timestamp, unit = 's')
submissions['day'] = submissions.date.dt.date

users_data = events.groupby('user_id', as_index= False)\
        .agg({'timestamp': 'max'})\
        .rename({'timestamp':'last_timestamp'}, axis = 'columns')
now = 1526772811
drop_out_threshold = 2592000

users_data['is_gone_user'] = (now - users_data.last_timestamp) > drop_out_threshold
users_scores = submissions.pivot_table(index='user_id',\
                        columns = 'submission_status',\
                        values = 'step_id',\
                        aggfunc = 'count',\
                        fill_value = 0)\
                        .reset_index()
users_data = users_data.merge(users_scores, on = 'user_id', how = 'outer')
users_data = users_data.fillna(0)
users_invent_data = events.pivot_table(index = 'user_id',
                        columns = 'action',
                        values  = 'step_id',
                        aggfunc = 'count',
                        fill_value = 0).reset_index()
users_data = users_data.merge(users_invent_data, how = 'outer')
users_days = events.groupby('user_id').day.nunique()
users_days.to_frame().reset_index()
users_data = users_data.merge(users_days, on = 'user_id', how = 'outer')
users_data['passed_course'] = users_data.passed > 170

In [104]:
users_data = users_data.merge(user_min_time, how = "outer")

In [105]:
users_data.head()

,user_id,last_timestamp,is_gone_user,correct,wrong,discovered,passed,started_attempt,viewed,day,passed_course,min_timestamp,user_learning_time_threshold
0,1,1472827464,True,0.0,0.0,1,0,0,1,1,False,1472827464,1_1473086664
1,2,1519226966,True,2.0,0.0,9,9,2,10,2,False,1514383364,2_1514642564
2,3,1444581588,True,29.0,23.0,91,87,30,192,7,False,1434358476,3_1434617676
3,5,1499859939,True,2.0,2.0,11,11,4,12,2,False,1466156809,5_1466416009
4,7,1521634660,True,0.0,0.0,1,1,0,1,1,False,1521634660,7_1521893860


In [40]:
events['user_time'] = events.user_id.map(str) + '_' + events.timestamp.map(str)

In [44]:
learning_time_threshold = 3* 24 * 60 * 60
learning_time_threshold

259200

In [51]:
user_learning_time_threshold = user_min_time.user_id.map(str) +'_'+(user_min_time.min_timestamp + learning_time_threshold).map(str)

In [53]:
user_min_time['user_learning_time_threshold'] = user_learning_time_threshold

In [54]:
user_min_time.head()

,user_id,min_timestamp,user_learning_time_threshold
0,1,1472827464,1_1473086664
1,2,1514383364,2_1514642564
2,3,1434358476,3_1434617676
3,5,1466156809,5_1466416009
4,7,1521634660,7_1521893860


In [56]:
events = events.merge(user_min_time[['user_id', 'user_learning_time_threshold']], how = 'outer')

In [57]:
events.head()

,step_id,timestamp,action,user_id,min_timestamp,date,day,user_time,user_learning_time_threshold
0,32815,1434340848,viewed,17632,1434340848,2015-06-15 04:00:48,2015-06-15,17632_1434340848,17632_1434600048
1,32815,1434340848,passed,17632,1434340848,2015-06-15 04:00:48,2015-06-15,17632_1434340848,17632_1434600048
2,32815,1434340848,discovered,17632,1434340848,2015-06-15 04:00:48,2015-06-15,17632_1434340848,17632_1434600048
3,32811,1434340895,discovered,17632,1434340848,2015-06-15 04:01:35,2015-06-15,17632_1434340895,17632_1434600048
4,32811,1434340895,viewed,17632,1434340848,2015-06-15 04:01:35,2015-06-15,17632_1434340895,17632_1434600048


In [58]:
events.shape

(3480703, 9)

In [59]:
events_train = events[events.user_time <= events.user_learning_time_threshold]

In [61]:
events_train.shape

(1014985, 9)

In [65]:
events_train.groupby('user_id').day.nunique().max()

4

In [66]:
submissions['user_time'] = submissions.user_id.map(str) +'_' + submissions.timestamp.map(str)
submissions = submissions.merge(user_min_time[['user_id', 'user_learning_time_threshold']], how = 'outer')
submissions_train = submissions[submissions.user_time <= submissions.user_learning_time_threshold]
submissions_train.groupby('user_id').day.nunique().max()

4

## X and y

In [150]:
X = submissions_train.groupby('user_id').day.nunique().to_frame().reset_index().rename(columns={'day':'days'})

In [151]:
steps_tried = submissions_train.groupby('user_id').step_id.nunique().to_frame().reset_index() \
                .rename(columns = {'step_id':'steps_tried'})

In [152]:
steps_tried.head()

,user_id,steps_tried
0,2,2
1,3,4
2,8,11
3,14,1
4,16,23


In [153]:
X = X.merge(steps_tried, on='user_id')

In [154]:
X = X.merge(submissions_train.pivot_table(index = 'user_id',\
                              columns = 'submission_status',\
                              values = 'step_id',\
                              aggfunc = 'count',\
                              fill_value = 0).reset_index())

In [155]:
X['correct_ratio'] = X.correct/(X.correct+X.wrong)

In [156]:
X = X.merge(events_train.pivot_table(index = 'user_id',\
                              columns = 'action',\
                              values = 'step_id',\
                              aggfunc = 'count',\
                              fill_value = 0).reset_index()[['user_id', 'viewed']], how = 'outer')

In [157]:
X.shape

(19234, 7)

In [158]:
X = X.fillna(0)

In [159]:
X = X.merge(users_data[['user_id', 'passed_course', 'is_gone_user']], how = 'outer')

In [160]:
X.head()

,user_id,days,steps_tried,correct,wrong,correct_ratio,viewed,passed_course,is_gone_user
0,2,1.0,2.0,2.0,0.0,1.00,9,False,True
1,3,1.0,4.0,4.0,4.0,0.50,20,False,True
2,8,1.0,11.0,9.0,21.0,0.30,154,False,True
3,14,1.0,1.0,0.0,1.0,0.00,9,False,True
4,16,3.0,23.0,23.0,27.0,0.46,132,True,True


In [161]:
X = X[~((X.is_gone_user == False) & (X.passed_course == False))]

In [162]:
X.head()

,user_id,days,steps_tried,correct,wrong,correct_ratio,viewed,passed_course,is_gone_user
0,2,1.0,2.0,2.0,0.0,1.00,9,False,True
1,3,1.0,4.0,4.0,4.0,0.50,20,False,True
2,8,1.0,11.0,9.0,21.0,0.30,154,False,True
3,14,1.0,1.0,0.0,1.0,0.00,9,False,True
4,16,3.0,23.0,23.0,27.0,0.46,132,True,True


In [163]:
X.groupby(['passed_course','is_gone_user']).user_id.count()

passed_course  is_gone_user
False          True            16560
True           False             141
               True             1284
Name: user_id, dtype: int64

In [164]:
y = X.passed_course.map(int)

In [165]:
X = X.drop(['passed_course', 'is_gone_user'], axis = 1)

In [166]:
X = X.set_index(X.user_id)
X = X.drop('user_id', axis=1)

In [167]:
X.head()

,days,steps_tried,correct,wrong,correct_ratio,viewed
user_id,,,,,,
2,1.0,2.0,2.0,0.0,1.00,9
3,1.0,4.0,4.0,4.0,0.50,20
8,1.0,11.0,9.0,21.0,0.30,154
14,1.0,1.0,0.0,1.0,0.00,9
16,3.0,23.0,23.0,27.0,0.46,132
